In [ ]:
from pyspark.sql import SparkSession, functions as F
import requests
import os

os.environ['PYSPARK_PYTHON'] = '/Users/jhollan/opt/miniconda3/envs/pyspark/bin/python'

In [ ]:
session = SparkSession.builder.appName("medium-tldr").getOrCreate()

In [ ]:
df = session.read.csv('medium_data.csv', header=True)
# df.show()
df_pd = df.toPandas().set_index('id').head(5)

In [ ]:
df_pd = df_pd.loc[~df_pd["url"].str.startswith('https://')]
df_pd.count()

In [ ]:
@F.udf(returnType=F.StringType())
def callAnalyze(url):
    res = requests.post('http://localhost:8080/', json={'article_url': url})
    return res.text

In [ ]:
df_tldr = df.select(callAnalyze(F.col('url')))

In [ ]:
df_tldr.toPandas().head(5)